In [1]:
import os, re
import numpy as np
import tensorflow as tf

file_path = os.getenv("HOME") + "/aiffel/lyricist/data/shakespeare.txt"
with open(file_path, 'r') as f:
    raw_corpus = f.read().splitlines()
    
print(raw_corpus[:9])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


In [3]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue
    if sentence[-1] == ':': continue
        
    if idx > 9: break
        
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


## 1. 문장부호
## 2. 대소문자 구별
## 3. 특수문자

In [9]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,¿])", r"\1", sentence)
    sentence = re.sub(r'[""]+', "", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+"," ",sentence)
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

print(preprocess_sentence('This @_is ;;; sample   sentence.'))

<start> this is sample sentence. <end>


In [10]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
corpus[:10]

['<start> before we proceed any further, hear me speak. <end>',
 '<start> speak, speak. <end>',
 '<start> you are all resolved rather to die than to famish? <end>',
 '<start> resolved. resolved. <end>',
 '<start> first, you know caius marcius is chief enemy to the people. <end>',
 '<start> we know t, we know t. <end>',
 '<start> let us kill him, and we ll have corn at our own price. <end>',
 '<start> is t a verdict? <end>',
 '<start> no more talking on t let it be done away, away! <end>',
 '<start> one word, good citizens. <end>']

In [12]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=7000,
    filters='',
    oov_token='<unk>'
    )
    
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  144   35 ...    0    0    0]
 [   2  521  492 ...    0    0    0]
 [   2   11   42 ...    0    0    0]
 ...
 [   2  137    1 ...    0    0    0]
 [   2   28   55 ...    0    0    0]
 [   2 1005   28 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f5d8bafda90>


In [15]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    
    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : the
5 : and
6 : i
7 : to
8 : of
9 : my
10 : a


In [17]:
src_input = tensor[:, :-1]

tgt_input = tensor[:, 1:]

print(src_input[0])
print(tgt_input[0])

[   2  144   35 1332  129 3602  124   29  492    3    0    0    0    0
    0    0    0]
[ 144   35 1332  129 3602  124   29  492    3    0    0    0    0    0
    0    0    0]


In [18]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 17), (256, 17)), types: (tf.int32, tf.int32)>

In [21]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

In [22]:
for src_sample, tgt_sample in dataset.take(1): break

model(src_sample)

<tf.Tensor: shape=(256, 17, 7001), dtype=float32, numpy=
array([[[ 2.96963583e-04, -9.84988073e-05,  1.68532002e-04, ...,
          2.74021004e-04,  3.53796378e-04, -4.75821798e-05],
        [ 2.97993800e-04, -2.78735883e-04,  8.22538350e-05, ...,
          5.64318383e-04,  1.06010621e-03, -1.13617156e-04],
        [ 1.64809317e-04, -2.08750644e-04,  2.41903792e-04, ...,
          8.62801273e-04,  1.19257183e-03, -3.12806864e-04],
        ...,
        [ 3.21184308e-03,  2.90896418e-03,  4.54097135e-05, ...,
         -1.92306296e-03,  3.01296677e-04,  1.46853016e-03],
        [ 3.68216168e-03,  3.54438461e-03, -1.22460851e-05, ...,
         -2.35810084e-03,  5.55778170e-05,  1.73927203e-03],
        [ 4.07445291e-03,  4.08856245e-03, -5.92359829e-05, ...,
         -2.72539840e-03, -1.86793710e-04,  1.94423751e-03]],

       [[ 2.96963583e-04, -9.84988073e-05,  1.68532002e-04, ...,
          2.74021004e-04,  3.53796378e-04, -4.75821798e-05],
        [ 6.12970965e-04,  6.88868167e-05, -7.

In [26]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
93/93 [==============================] - 31s 304ms/step - loss: 4.6019
Epoch 2/30
93/93 [==============================] - 28s 306ms/step - loss: 3.0195
Epoch 3/30
93/93 [==============================] - 29s 307ms/step - loss: 2.9182
Epoch 4/30
93/93 [==============================] - 29s 308ms/step - loss: 2.8408
Epoch 5/30
93/93 [==============================] - 29s 309ms/step - loss: 2.7828
Epoch 6/30
93/93 [==============================] - 29s 308ms/step - loss: 2.7196
Epoch 7/30
93/93 [==============================] - 29s 309ms/step - loss: 2.6666
Epoch 8/30
93/93 [==============================] - 29s 309ms/step - loss: 2.6159
Epoch 9/30
93/93 [==============================] - 29s 309ms/step - loss: 2.5584
Epoch 10/30
93/93 [==============================] - 29s 309ms/step - loss: 2.5202
Epoch 11/30
93/93 [==============================] - 29s 309ms/step - loss: 2.4576
Epoch 12/30
93/93 [==============================] - 29s 309ms/step - loss: 2.4133
Epoch 13/30
9

In [30]:
def generate_text(model, tokenizer, init_sentence='<start>', max_len=20):
    # 테스트를 위해 입력받은 init_sentence도 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index['<end>']
    
    # 단어 하나씩 예측해 문장을 만듭니다
    # 1. 입력받은 문장의 텐서를 입력
    # 2. 예측된 값 중 가장 높은 확률 -> word index
    # 3. 2에서 예측된 word index를 문장 뒤에 붙ㅁ
    # 4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성 종료
    while True:
        # 1
        predict = model(test_tensor)
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]
        # 3
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break
            
    generated = ''
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + ""
        
    return generated

In [31]:
generate_text(model, tokenizer, init_sentence='<start>')

'<start>and<unk><unk>to<unk><end>'